# Create a master frame for plotting anything with seaborn.
Create a file `worm_data_master.csv` that contains that aggregates the worm data from all datasets.

In [ ]:
from data._utils import load_dataset
import pandas as pd
from utils import VALID_DATASETS

In [3]:
all_worm_datasets = [
    load_dataset(dataset)
    for dataset in VALID_DATASETS
    if not dataset.__contains__("sine")
]

In [4]:
new_dict = dict()

for dataset in all_worm_datasets:
    for worm in dataset:
        for neuron, slot in dataset[worm]["neuron_to_slot"].items():
            # get data to fill in table
            dataset_name = dataset[worm]["dataset"]
            smooth_method = dataset[worm]["smooth_method"]
            calcium_data = dataset[worm]["calcium_data"][:, slot].numpy()
            smooth_calcium_data = dataset[worm]["smooth_calcium_data"][:, slot].numpy()
            residual_calcium = dataset[worm]["residual_calcium"][:, slot].numpy()
            smooth_residual_calcium = dataset[worm]["smooth_residual_calcium"][
                :, slot
            ].numpy()
            time_in_seconds = dataset[worm]["time_in_seconds"].squeeze().numpy()
            dt = dataset[worm]["dt"].squeeze().numpy()
            max_timesteps = dataset[worm]["max_timesteps"]
            if neuron in dataset[worm]["named_neuron_to_slot"]:
                is_named_neuron = True
                is_unknown_neuron = False
            elif neuron in dataset[worm]["unknown_neuron_to_slot"]:
                is_named_neuron = False
                is_unknown_neuron = True
            else:
                print("Neither named nor unknown neuron?!?!?!")

            # fill in table
            new_dict.setdefault("dataset", new_dict.get("dataset", [])).append(
                dataset_name
            )
            new_dict.setdefault("worm", new_dict.get("worm", [])).append(worm)
            new_dict.setdefault("neuron", new_dict.get("neuron", [])).append(neuron)
            new_dict.setdefault("slot", new_dict.get("slot", [])).append(slot)
            new_dict.setdefault(
                "is_named_neuron", new_dict.get("is_named_neuron", [])
            ).append(is_named_neuron)
            new_dict.setdefault(
                "is_unknown_neuron", new_dict.get("is_unknown_neuron", [])
            ).append(is_unknown_neuron)
            new_dict.setdefault(
                "calcium_data", new_dict.get("calcium_data", [])
            ).append(calcium_data)
            new_dict.setdefault(
                "smooth_calcium_data", new_dict.get("smooth_calcium_data", [])
            ).append(smooth_calcium_data)
            new_dict.setdefault(
                "residual_calcium", new_dict.get("residual_calcium", [])
            ).append(residual_calcium)
            new_dict.setdefault(
                "smooth_residual_calcium", new_dict.get("smooth_residual_calcium", [])
            ).append(smooth_residual_calcium)
            new_dict.setdefault(
                "smooth_method", new_dict.get("smooth_method", [])
            ).append(smooth_method)
            new_dict.setdefault(
                "time_in_seconds", new_dict.get("time_in_seconds", [])
            ).append(time_in_seconds)
            new_dict.setdefault("dt", new_dict.get("dt", [])).append(dt)
            new_dict.setdefault(
                "max_timesteps", new_dict.get("max_timesteps", [])
            ).append(max_timesteps)

In [5]:
df = pd.DataFrame.from_dict(new_dict)
pd.DataFrame.to_csv(df, "../data/worm_data_master.csv")

In [10]:
df.head(300)

,dataset,worm,neuron,slot,is_named_neuron,is_unknown_neuron,calcium_data,smooth_calcium_data,residual_calcium,smooth_residual_calcium,smooth_method,time_in_seconds,dt,max_timesteps
0,Kaplan2020,worm0,ADAL,0,True,False,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",FFT,"[0.0, 0.33, 0.66, 0.99, 1.32, 1.65, 1.98, 2.31...","[0.0, 0.33, 0.33, 0.32999998, 0.33000004, 0.32...",5455
1,Kaplan2020,worm0,55,1,False,True,"[0.9523473, 0.96172035, 0.89560115, 0.937194, ...","[0.15089218, 0.3194357, 0.4770283, 0.6188751, ...","[0.0, 0.009373069, -0.066119194, 0.041592836, ...","[0.0, 0.1685435, 0.15759262, 0.14184678, 0.122...",FFT,"[0.0, 0.33, 0.66, 0.99, 1.32, 1.65, 1.98, 2.31...","[0.0, 0.33, 0.33, 0.32999998, 0.33000004, 0.32...",5455
2,Kaplan2020,worm0,101,2,False,True,"[0.15257692, 0.15386257, 0.10407905, 0.1477859...","[-0.2801806, -0.18687333, -0.09690169, -0.0133...","[0.0, 0.0012856424, -0.049783513, 0.043706864,...","[0.0, 0.09330727, 0.08997164, 0.083529145, 0.0...",FFT,"[0.0, 0.33, 0.66, 0.99, 1.32, 1.65, 1.98, 2.31...","[0.0, 0.33, 0.33, 0.32999998, 0.33000004, 0.32...",5455
3,Kaplan2020,worm0,ADER,3,True,False,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",FFT,"[0.0, 0.33, 0.66, 0.99, 1.32, 1.65, 1.98, 2.31...","[0.0, 0.33, 0.33, 0.32999998, 0.33000004, 0.32...",5455
4,Kaplan2020,worm0,ADFL,4,True,False,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",FFT,"[0.0, 0.33, 0.66, 0.99, 1.32, 1.65, 1.98, 2.31...","[0.0, 0.33, 0.33, 0.32999998, 0.33000004, 0.32...",5455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,Kaplan2020,worm0,VD3,295,True,False,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",FFT,"[0.0, 0.33, 0.66, 0.99, 1.32, 1.65, 1.98, 2.31...","[0.0, 0.33, 0.33, 0.32999998, 0.33000004, 0.32...",5455
296,Kaplan2020,worm0,VD4,296,True,False,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",FFT,"[0.0, 0.33, 0.66, 0.99, 1.32, 1.65, 1.98, 2.31...","[0.0, 0.33, 0.33, 0.32999998, 0.33000004, 0.32...",5455
297,Kaplan2020,worm0,87,297,False,True,"[-0.71460426, -0.7624105, -0.7254513, -0.76921...","[-0.804421, -0.7948413, -0.7858737, -0.7777533...","[0.0, -0.047806263, 0.03695923, -0.04376638, 0...","[0.0, 0.009579718, 0.008967578, 0.008120418, 0...",FFT,"[0.0, 0.33, 0.66, 0.99, 1.32, 1.65, 1.98, 2.31...","[0.0, 0.33, 0.33, 0.32999998, 0.33000004, 0.32...",5455
298,Kaplan2020,worm0,VD6,298,True,False,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",FFT,"[0.0, 0.33, 0.66, 0.99, 1.32, 1.65, 1.98, 2.31...","[0.0, 0.33, 0.33, 0.32999998, 0.33000004, 0.32...",5455


In [11]:
df.tail(300)

,dataset,worm,neuron,slot,is_named_neuron,is_unknown_neuron,calcium_data,smooth_calcium_data,residual_calcium,smooth_residual_calcium,smooth_method,time_in_seconds,dt,max_timesteps
46208,Nguyen2017,worm2,79,2,False,True,"[-0.45696837, -0.45696837, -0.45696837, -0.456...","[-0.56521416, -0.5880209, -0.620382, -0.660049...","[0.0, 0.0, 0.0, 0.0, -0.27932322, -0.021114886...","[0.0, -0.022806764, -0.03236109, -0.039667487,...",FFT,"[0.0, 0.16999994, 0.3319999, 0.50299984, 0.664...","[0.0, 0.16999994, 0.16199996, 0.17099994, 0.16...",2849
46209,Nguyen2017,worm2,ADER,3,True,False,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",FFT,"[0.0, 0.16999994, 0.3319999, 0.50299984, 0.664...","[0.0, 0.16999994, 0.16199996, 0.17099994, 0.16...",2849
46210,Nguyen2017,worm2,ADFL,4,True,False,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",FFT,"[0.0, 0.16999994, 0.3319999, 0.50299984, 0.664...","[0.0, 0.16999994, 0.16199996, 0.17099994, 0.16...",2849
46211,Nguyen2017,worm2,98,5,False,True,"[-0.29725155, -0.29725155, -0.29725155, -0.297...","[-0.3976849, -0.33690703, -0.27018398, -0.2008...","[0.0, 0.0, 0.0, 0.0, 0.07516524, 0.23127185, 0...","[0.0, 0.060777873, 0.06672305, 0.06933616, 0.0...",FFT,"[0.0, 0.16999994, 0.3319999, 0.50299984, 0.664...","[0.0, 0.16999994, 0.16199996, 0.17099994, 0.16...",2849
46212,Nguyen2017,worm2,ADLL,6,True,False,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",FFT,"[0.0, 0.16999994, 0.3319999, 0.50299984, 0.664...","[0.0, 0.16999994, 0.16199996, 0.17099994, 0.16...",2849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46503,Nguyen2017,worm2,VD5,297,True,False,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",FFT,"[0.0, 0.16999994, 0.3319999, 0.50299984, 0.664...","[0.0, 0.16999994, 0.16199996, 0.17099994, 0.16...",2849
46504,Nguyen2017,worm2,VD6,298,True,False,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",FFT,"[0.0, 0.16999994, 0.3319999, 0.50299984, 0.664...","[0.0, 0.16999994, 0.16199996, 0.17099994, 0.16...",2849
46505,Nguyen2017,worm2,VD7,299,True,False,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",FFT,"[0.0, 0.16999994, 0.3319999, 0.50299984, 0.664...","[0.0, 0.16999994, 0.16199996, 0.17099994, 0.16...",2849
46506,Nguyen2017,worm2,42,300,False,True,"[-0.24859065, -0.24859065, -0.24859065, -0.227...","[-0.20505077, -0.24610923, -0.2959029, -0.3505...","[0.0, 0.0, 0.0, 0.021114588, -0.017434478, -0....","[0.0, -0.041058466, -0.049793676, -0.05467871,...",FFT,"[0.0, 0.16999994, 0.3319999, 0.50299984, 0.664...","[0.0, 0.16999994, 0.16199996, 0.17099994, 0.16...",2849
